In [ ]:
import numpy as np
import cv2

In [ ]:
cap = cv2.VideoCapture('proj2_v2.mp4')

In [ ]:
def convolve(input_image, kernel, padding = 0, stride = 1):
    assert (stride > 0), "stride cannot be less than 1"

    input_image = input_image.reshape(input_image.shape[0], input_image.shape[1], -1) # to handle gray scale images
    h, w, c = input_image.shape
    kernel_size = kernel.shape[0]
    if kernel_size % 2 == 0: #even kernel
        print("Error! The kernel cannot be even")
        return -1
    if kernel.shape[0] is not kernel.shape[1]:
        print("Error! The kernel is not square")
        return -1

    # create padded image
    padded_input_image = np.zeros((h + 2*padding,  w + 2*padding, c))
    padded_input_image[padding: padding + h, padding: padding + w, :] = input_image
    #print("Original input image shape : ", input_image.shape)
    #print("Padded input image shape : ", padded_input_image.shape)

    # Perform the gradient
    h_out = int((h + padding * 2 - kernel_size) / stride) + 1
    w_out = int((w + padding * 2 - kernel_size) / stride) + 1
    conv_image = np.zeros((h_out, w_out, c))
    for i in range(h_out):
        for j in range(w_out):
            for k in range(c):
                conv_image[i, j, k] = np.sum(padded_input_image[i*stride : (i*stride + kernel_size), \
                                            j*stride : (j*stride + kernel_size), k] * kernel)

    #print("Convoluted output size : ", conv_image.shape)
    return conv_image

In [ ]:
def process_frame(frame):
    # Step 1: Check for blurry frames using Variance of the Laplacian
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # gImg=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    #cv2_imshow(gray)
    kernel = np.array([[0, 1, 0], [1, -4, 1], [0, 1, 0]])
    vImg=convolve(gray, kernel, padding = 0, stride = 1)
    #cv2_imshow(vImg)
    vMean=np.mean(vImg)
    #print(vMean)
    vImg-=vMean
    #cv2_imshow(vImg)
    vImg = vImg.astype(np.uint8)
    #print(vImg)
    h, w, c = vImg.shape
    #print(h,w,c)
    varience = sum(sum(vImg))
    #print(varience)
    if(varience < 150):
      print("Varience low", varience)
      return None
    #   cv2_imshow(frame)
    # laplacian_var = cv2.Laplacian(gray, cv2.CV_64F).var()

    # if laplacian_var < 150:
    #     return None  # Skip blurry frames

    # Step 2: Convert to grayscale and threshold to segment unwanted background
    # gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = gray[:480,:600]
    kernel_size = 3
    blur_gray = cv2.medianBlur(gray,kernel_size)
    # _, threshHough = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)
    _, thresh = cv2.threshold(blur_gray, 200, 255, cv2.THRESH_BINARY)
    #cv2_imshow(thresh)
    # Step 3: Detect edges using Canny edge detector
    edges = cv2.Canny(thresh, 150, 250, apertureSize=3)
    h, w = edges.shape[:2]
    mask = np.zeros((h+2, w+2), np.uint8)
    # cv2_imshow(edges)
    # cv2_imshow(blur_gray)
    cv2.floodFill(edges, mask, (0,0), 123)
    floodfill = edges.copy()
    # cv2_imshow(floodfill)
    bg = np.zeros_like(edges)
    # cv2_imshow(bg)
    bg[edges == 123] = 255
    # cv2_imshow(bg)
    #bg = cv2.blur(bg, (3,3))
    #cv2_imshow(bg)
    edgesMask = cv2.Canny(bg, 100, 250, apertureSize=3)
    # cv2_imshow(edgesMask)
    # Step 4: Use Hough Transform to extract straight lines
    # Step 5: Filter out short lines and keep dominant ones
    lines = cv2.HoughLinesP(edgesMask,rho = 1,theta = 1*np.pi/180,threshold = 90,minLineLength = 100,maxLineGap = 50)

    # print(lines)
    print(lines)
    # Step 6: Compute intersections of Hough Lines
    valid_corners = []
    output_frame = frame.copy()
    for line in lines:
        x1 = line[0][0]
        y1 = line[0][1]
        x2 = line[0][2]
        y2 = line[0][3]
        cv2.line(output_frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
        valid_corners.append((x1,y1))
        valid_corners.append((x2,y2))
        valid_corners.append((x1-1,y1-1))
        valid_corners.append((x2-1,y2-1))
        valid_corners.append((x1+1,y1+1))
        valid_corners.append((x2+1,y2+1))
        valid_corners.append((x1-1,y1+1))
        valid_corners.append((x2-1,y2+1))
        valid_corners.append((x1+1,y1-1))
        valid_corners.append((x2+1,y2-1))
    #cv2_imshow(output_frame)
    # Step 7: Filter out extraneous corners
    corners = cv2.goodFeaturesToTrack(bg, 100, 0.02, 100)
    corners2 = cv2.cornerHarris(bg, 2, 7, 0.01)
    cnt = corners2 > 0.01 * corners2.max()
    # print(np.where(cnt==True))
    # print(corners2)\
    # corners = corners.tolist()
    # for i in corners:
    #   corners.append([[i[0][0]-1,i[0][1]-1]])
    #   corners.append([[i[0][0]+1,i[0][1]+1]])
    #   corners.append([[i[0][0]-1,i[0][1]+1]])
    #   corners.append([[i[0][0]+1,i[0][1]-1]])
    corners = np.intp(corners)
    print(corners)

    # output_frame[corners2 > 0.01 * corners2.max()] = [0,255,0]

    # draw red color circles on all corners
    for i in valid_corners:
        if i in corners:
          cv2.circle(output_frame, i, 3, (0, 0, 225), -1)

    # cv2_imshow(output_frame)
    return output_frame

In [ ]:
cap.set(1, 35)
ret, polyFrame = cap.read()
cv2.imshow("",polyFrame)

In [ ]:
# Applying the image processing for specific image.
newFrame = process_frame(polyFrame)
cv2.imshow("",newFrame)

In [ ]:
fps = 10
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Create VideoWriter object for output video
out = cv2.VideoWriter('hkyatham_project2.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
count = 1
while cap.isOpened():
    ret, frame = cap.read()
    print(count)
    count+=1
    if not ret:
        break

    processed_frame = process_frame(frame)

    if processed_frame is not None:
        out.write(processed_frame)
cap.release()
out.release()
cv2.destroyAllWindows()